In [3]:
'''
https://www.kaggle.com/code/itsuki9180/rsna2024-lsdc-making-dataset
'''

'\nhttps://www.kaggle.com/code/itsuki9180/rsna2024-lsdc-making-dataset\n'

In [4]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # 필요한 만큼만 메모리를 사용하도록 설정
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            
        # 특정 GPU에 연산을 할당
        tf.config.set_visible_devices(gpus[0], 'GPU')
        
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
        
    except RuntimeError as e:
        print(e)


1 Physical GPUs, 1 Logical GPU


In [5]:
import pydicom
import glob, os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
import re

In [6]:
rd = 'spine'

In [7]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [24]:
dfc = pd.read_csv(f'{rd}/train_label_coordinates.csv')
dfc

,study_id,series_id,instance_number,condition,level,x,y
0,4003253,702807833,8,Spinal Canal Stenosis,L1/L2,322.831858,227.964602
1,4003253,702807833,8,Spinal Canal Stenosis,L2/L3,320.571429,295.714286
2,4003253,702807833,8,Spinal Canal Stenosis,L3/L4,323.030303,371.818182
3,4003253,702807833,8,Spinal Canal Stenosis,L4/L5,335.292035,427.327434
4,4003253,702807833,8,Spinal Canal Stenosis,L5/S1,353.415929,483.964602
...,...,...,...,...,...,...,...
48687,4290709089,4237840455,11,Left Neural Foraminal Narrowing,L1/L2,219.465940,97.831063
48688,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L2/L3,205.340599,140.207084
48689,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L3/L4,202.724796,181.013624
48690,4290709089,4237840455,12,Left Neural Foraminal Narrowing,L4/L5,202.933333,219.733333


In [9]:
df = pd.read_csv(f'{rd}/train_series_descriptions.csv')
df.head()

,study_id,series_id,series_description
0,4003253,702807833,Sagittal T2/STIR
1,4003253,1054713880,Sagittal T1
2,4003253,2448190387,Axial T2
3,4646740,3201256954,Axial T2
4,4646740,3486248476,Sagittal T1


In [13]:
# 각 study_id에 대해 3~6개의 series_id를 관찰할 수 있습니다.

In [14]:
df['series_description'].value_counts()

series_description
Axial T2            2340
Sagittal T1         1980
Sagittal T2/STIR    1974
Name: count, dtype: int64

In [15]:
# 4개 이상의 series_id를 가진 대부분의 study_id에는 2개 이상의 Axial T2가 있습니다.

In [16]:
df[df['study_id']==4096820034]

,study_id,series_id,series_description
5997,4096820034,300517765,Axial T2
5998,4096820034,2097107888,Axial T2
5999,4096820034,2602265508,Sagittal T2/STIR
6000,4096820034,2679683906,Axial T2
6001,4096820034,3114813181,Axial T2
6002,4096820034,3236751045,Sagittal T1


In [17]:
dfc[dfc['study_id']==4096820034]

,study_id,series_id,instance_number,condition,level,x,y
46392,4096820034,300517765,5,Right Subarticular Stenosis,L4/L5,305.500000,372.444444
46393,4096820034,300517765,11,Right Subarticular Stenosis,L5/S1,298.388889,358.222222
46394,4096820034,2097107888,10,Left Subarticular Stenosis,L1/L2,348.379062,378.070442
46395,4096820034,2097107888,16,Left Subarticular Stenosis,L2/L3,344.193491,369.699299
46396,4096820034,2097107888,22,Left Subarticular Stenosis,L3/L4,342.100705,359.235371
46397,4096820034,2602265508,7,Spinal Canal Stenosis,L1/L2,268.706316,88.525822
46398,4096820034,2602265508,7,Spinal Canal Stenosis,L2/L3,257.288476,135.399061
46399,4096820034,2602265508,7,Spinal Canal Stenosis,L3/L4,247.673452,180.469484
46400,4096820034,2602265508,7,Spinal Canal Stenosis,L4/L5,244.668757,220.732394
46401,4096820034,2602265508,7,Spinal Canal Stenosis,L5/S1,253.081903,260.394366


In [18]:
'''
다음과 같이 대응됨을 알 수 있습니다:

Axial T2 => (Left|Right) Subarticular Stenosis (10 classes)
Sagittal T2/STIR => Spinal Canal Stenosis (5 Classes)
Sagittal T1 => (Left|Right) Neural Foraminal Narrowing (10 classes)


Axial T2를 잘 고기 어렵다고 판단되어 교육 중 무작위로 선택하기로 했습니다. 
나머지 2개는 동일한 간격으로 이미지를 저장할 예정입니다.

'''

'\n다음과 같이 대응됨을 알 수 있습니다:\n\nAxial T2 => (Left|Right) 관절하 협착 (10개 클래스)\nSagittal T2/STIR =>척수관협착증 (5급)\nSagittal T1 => (좌|우) 신경포자막 협착 (10개 클래스)\nAxial T2를 잘 고기 어렵다고 판단되어 교육 중 무작위로 선택하기로 했습니다. \n나머지 2개는 동일한 간격으로 이미지를 저장할 예정입니다.\n\n'

In [19]:
'''
dcm에서 png 내보내기
.dcm 포맷 파일은 다양한 픽셀 값과 이미지 모양을 가지고 있습니다. 
딥러닝 프레임워크에서 사용하기 위해 값이 일정 범위에 속하도록 하고 모양의 크기를 512px로 조정합니다.
'''

'\ndcm에서 png 내보내기\n.dcm 포맷 파일은 다양한 픽셀 값과 이미지 모양을 가지고 있습니다. \n딥러닝 프레임워크에서 사용하기 위해 값이 일정 범위에 속하도록 하고 모양의 크기를 512px로 조정합니다.\n'

In [20]:
def imread_and_imwirte(src_path, dst_path):
    dicom_data = pydicom.dcmread(src_path)
    image = dicom_data.pixel_array
    image = (image - image.min()) / (image.max() - image.min() +1e-6) * 255
    img = cv2.resize(image, (512, 512),interpolation=cv2.INTER_CUBIC)
    assert img.shape==(512,512)
    cv2.imwrite(dst_path, img)

In [21]:
st_ids = df['study_id'].unique()
st_ids[:3], len(st_ids)

(array([4003253, 4646740, 7143189], dtype=int64), 1975)

In [22]:
desc = list(df['series_description'].unique())
desc

['Sagittal T2/STIR', 'Sagittal T1', 'Axial T2']

In [23]:
for idx, si in enumerate(tqdm(st_ids, total=len(st_ids))):
    pdf = df[df['study_id']==si]
    for ds in desc:
        ds_ = ds.replace('/', '_')
        pdf_ = pdf[pdf['series_description']==ds]
        os.makedirs(f'cvt_png/{si}/{ds_}', exist_ok=True)
        allimgs = []
        for i, row in pdf_.iterrows():
            pimgs = glob.glob(f'{rd}/train_images/{row["study_id"]}/{row["series_id"]}/*.dcm')
            pimgs = sorted(pimgs, key=natural_keys)
            allimgs.extend(pimgs)
            
        if len(allimgs)==0:
            print(si, ds, 'has no images')
            continue

        if ds == 'Axial T2':
            for j, impath in enumerate(allimgs):
                dst = f'cvt_png/{si}/{ds}/{j:03d}.png'
                imread_and_imwirte(impath, dst)
                
        elif ds == 'Sagittal T2/STIR':
            
            step = len(allimgs) / 10.0
            st = len(allimgs)/2.0 - 4.0*step
            end = len(allimgs)+0.0001
            for j, i in enumerate(np.arange(st, end, step)):
                dst = f'cvt_png/{si}/{ds_}/{j:03d}.png'
                ind2 = max(0, int((i-0.5001).round()))
                imread_and_imwirte(allimgs[ind2], dst)
                
            assert len(glob.glob(f'cvt_png/{si}/{ds_}/*.png'))==10
                
        elif ds == 'Sagittal T1':
            step = len(allimgs) / 10.0
            st = len(allimgs)/2.0 - 4.0*step
            end = len(allimgs)+0.0001
            for j, i in enumerate(np.arange(st, end, step)):
                dst = f'cvt_png/{si}/{ds}/{j:03d}.png'
                ind2 = max(0, int((i-0.5001).round()))
                imread_and_imwirte(allimgs[ind2], dst)
                
            assert len(glob.glob(f'cvt_png/{si}/{ds}/*.png'))==10

 57%|████████████████████████████████████████████▍                                 | 1126/1975 [20:55<14:00,  1.01it/s]

2492114990 Sagittal T1 has no images


 64%|██████████████████████████████████████████████████▏                           | 1272/1975 [23:30<11:13,  1.04it/s]

2780132468 Sagittal T1 has no images


 70%|██████████████████████████████████████████████████████▍                       | 1378/1975 [25:28<09:19,  1.07it/s]

3008676218 Sagittal T2/STIR has no images


100%|██████████████████████████████████████████████████████████████████████████████| 1975/1975 [36:23<00:00,  1.11s/it]
